# Metrics in Context | A Proof of Concept

### Describing Citation Data with [Frictionless](https://frictionlessdata.io/)

*Asura Enkhbayar, 04.02.2021*

---

This notebook presents a proof of concept for a standardized approach to citation data and, more broadly, scholarly metrics. To do so, we will take a bibliometric dataset consisting of four spreadsheets and build a Frictionless [Data Package](https://specs.frictionlessdata.io/data-package/) using a systematic approach to describe each tables provenance. I will first introduce the dataset at hand and provide an quick overview and situate it in the context of traditional bibliometric work. Then, I will step-by-step introduce a new conceptual framework for scholarly metrics and gradually incorporate the dataset into the final data package. Finally, a few concluding remarks and a brief discussion of the potential and limitations of this approach.

*Note: For a broader overview of the project please refer to the [README](https://github.com/Bubblbu/metrics-in-context) in the project repository.*

<img style="float: center;" width=25%, src="../../materials/assets/fosdem2021.jpg">

## Let's have a look at our input dataset

For the purpose of this particular notebook, I will not go into the details of how this particular dataset was queried for and created. Of course, providing that kind of information would be part of the aims of developing a Frictionless Data Package for scholarly metrics. For now, I will simply describe the spreadsheets at hand and provide some  cursory descriptions of its origins.

The dataset we are working with consists of four spreadsheets that were created based on data provided by [Scite.ai](https://scite.ai/). The particular sample was created by querying for recent articles indexed in Pubmed on Amyotrophic lateral sclerosis (ALS). Scite is one of the newer data sources that not only provide the binary citation link between documents, but attempt to extract what has been established as citation contexts. While the traditional citation count relies on the datafication of bibliographies and reference lists, context-aware citations extract each individual in-text mention of articles.



- `contexts.csv`: The contexts of the events captured which typically include article metadata. 
- `traces.csv`: The individual traced citational links between documents.
- `citations.csv`: This table contains patterns derived from the traces on the level of each individual citation pair.
- `articles.csv`: A table with article-level metrics.

In [15]:
from pprint import pprint
from pathlib import Path

import pandas as pd
from frictionless import describe_schema

# Setup constants
input_files = Path("./input_files")

In [16]:
pd.read_csv(input_files / "contexts.csv").head()

,doi,slug,type,title,abstract,authors,keywords,year,shortJournal,publisher,issue,volume,page,retracted,memberId,issns,editorialNotices,journalSlug,journal,rwStatus
0,10.1371/journal.pone.0159593,shared-genetic-factors-involved-in-j81d1l,journal-article,Shared Genetic Factors Involved in Celiac Dise...,Background and ObjectivesGenome-wide associati...,"[{'family': 'Mostowy', 'given': 'Joanna'}, {'f...",['Research Article'],2016.0,PLoS ONE,Public Library of Science (PLoS),8,11,e0159593,False,340,['1932-6203'],[],plos-one-lZbEk,Plos One,NaN
1,10.1080/21541248.2016.1276999,molecular-control-of-rab-activity-kpJgay,journal-article,"Molecular control of Rab activity by GEFs, GAP...",ABSTRACTRab proteins are the major regulators ...,"[{'family': 'Müller', 'given': 'Matthias P.'},...",['Review'],2017.0,Small GTPases,Informa UK Limited,1-2,9,5-21,False,301,"['2154-1248', '2154-1256']",[],small-gtpases-YZrGD,Small GTPases,NaN
2,10.1101/gad.281030.116,codon-usage-affects-the-structure-Zwy9Zp,journal-article,Codon usage affects the structure and function...,\nCodon usage bias is a universal feature of a...,"[{'family': 'Fu', 'given': 'Jingjing'}, {'fami...","['[Keywords: circadian clock', 'codon usage', ...",2016.0,Genes Dev.,Cold Spring Harbor Laboratory,15,30,1761-1775,False,246,"['0890-9369', '1549-5477']",[],genes-development-6828p,Genes & Development,NaN
3,10.3389/fnins.2016.00235,positron-emission-tomography-reveals-abnormal-...,journal-article,Positron Emission Tomography Reveals Abnormal ...,Recent studies have demonstrated alterations i...,"[{'family': 'Qiu', 'given': 'Xiangzhe'}, {'fam...",['Neuroscience'],2016.0,Front. Neurosci.,Frontiers Media SA,NaN,10,NaN,False,1965,['1662-453X'],[],frontiers-in-neuroscience-RVvG4,Frontiers in Neuroscience,NaN
4,10.1007/s00439-016-1683-5,the-hnrnp-family-insights-into-V23EgE,journal-article,The hnRNP family: insights into their role in ...,Heterogeneous nuclear ribonucleoproteins (hnRN...,"[{'family': 'Geuens', 'given': 'Thomas'}, {'fa...",['Review'],2016.0,Hum Genet,Springer Science and Business Media LLC,8,135,851-867,False,297,"['0340-6717', '1432-1203']",[],human-genetics-vJNKJ,Human Genetics,NaN


## A Conceptual Framework for Scholarly Metrics

Here, I want to attempt to briefly introduce the conceptual framework used to systematize scholarly metrics. This framework is part of a broader doctoral research project and is built around the philosophical shift from object-centrism to process-centrism. This shift from representation towards performance can also be expressed for scholarly metrics by emphasising the processes and practices that lead to outputs and citation counts. But how do these processes look like and how can we start to systematically describe them?

The usual story goes something like this: A *citation* links two scholarly texts expressed through an in-text mention and a bibliographic reference. This state is then captured by a *citation link* in their databases. This data then enables the creation of *citation metrics*. I would like to retell this story with some new vocabulary and some closer attention paid to the individual steps:

### Citational Events and Their Contexts

> A *citation* links two *scholarly texts* expressed through an in-text mention and a bibliographic reference.

Firstly, I propose to question the idea of the citation as one consistent concept itself. It is typically understood as a directional binary property between two peer-reviewed research articles. However, more and more citation databases are using text-processing methods to extract the individual in-text mentions with their contexts and furthermore the types of documents hosting citations is changing (e.g., datasets, software, mentions in social media). To accomodate for these changes, I suggest to think in terms of *citational events* and their *contexts*.

**Citational Events**

The typical citation is a statement in a peer-reviewed scholarly article referencing another piece of scholarly writing. Some of the new social (and technical) challenges around citations stem from changing citational practices such as the citing of preprints or datasets. So far, these changes have been pragmatically addressed by introduction of norms and dedicated infrastructure (e.g., the push for DOIs for datasets and iniatives that track their usage) but we are lacking the conceptual language to capture all these forms of citations.

Thus, I propose the understanding of the citation as an interdiscursive event (Nakassis, 2013).  

**Contexts**



### The Tracing of Events

> This state is then captured by a *citation link* in their databases.

How are events and their contexts turned into traces?

**Traces**



### Patterns and Metrics

> This data then enables the creation of *citation metrics*.



**Patterns**

All metrics are patterns, but not all patterns are metrics.

## Implementation

### Events

### Contexts

### Traces

### Patterns

#### Citation level patterns

This table contains citation patterns aggregated of individual citations.

Each table will thus contain:

- A unique ID
- A source DOI for the citing article
- A target DOI for the cited article
- One or more patterns
    - Each pattern also contain the form of aggregation and input traces/patterns
        - Optional for each input trace/pattern: a data resource

In [31]:
f = data_dir / "citation_patterns.csv"
cit_patterns = pd.read_csv(f)
cp_schema = describe_schema(f)
cp_schema.fields

[{'name': 'source', 'type': 'string'},
 {'name': 'target', 'type': 'string'},
 {'name': 'total_source_mentions', 'type': 'integer'},
 {'name': 'total_source_refs', 'type': 'integer'},
 {'name': 'mentions', 'type': 'integer'},
 {'name': 'norm_refs', 'type': 'number'},
 {'name': 'norm_mentions', 'type': 'number'},
 {'name': 'mentions_per_ref', 'type': 'number'},
 {'name': 'wf1', 'type': 'number'},
 {'name': 'wf2', 'type': 'number'},
 {'name': 'wf3', 'type': 'number'}]

In [29]:
cp_schema.name = "citation_patterns"
cp_schema.description = 

ap_schema.primary_key = "doi"
ap_schema.get_field("doi").title = "DOI of citing article"

{'fields': [{'name': 'source', 'type': 'string'},
  {'name': 'target', 'type': 'string'},
  {'name': 'total_source_mentions', 'type': 'integer'},
  {'name': 'total_source_refs', 'type': 'integer'},
  {'name': 'mentions', 'type': 'integer'},
  {'name': 'norm_refs', 'type': 'number'},
  {'name': 'norm_mentions', 'type': 'number'},
  {'name': 'mentions_per_ref', 'type': 'number'},
  {'name': 'wf1', 'type': 'number'},
  {'name': 'wf2', 'type': 'number'},
  {'name': 'wf3', 'type': 'number'}]}

### Article level patterns

This table contains citation patterns aggregated on the article level.

Each table will thus contain:

- A unique DOI
- One or more patterns for each DOI
    - Each pattern also contain the form of aggregation and input traces/patterns
        - Optional for each input trace/pattern: a data resource

In [6]:
f = "article_patterns.csv"
article_patterns = pd.read_csv(f)
ap_schema = describe_schema(f)

In [7]:
# Set primary key
ap_schema.primary_key = "doi"
ap_schema.get_field("doi").title = "DOI of citing article"

# First pattern: mentions_agg
mentions_agg = ap_schema.get_field("mentions_agg")
mentions_agg.title = "Aggregated mentions"
mentions_agg.description = "Sum of all incoming mentions for this DOI"
mentions_agg.type = "integer"
mentions_agg.missing_values = ["", "n/a", "NaN"]
mentions_agg.mic = {
    "type": "pattern",
    "prov": {
        "operation": {
            "description": "Aggregation of all citations by DOI"
            "type": "aggregation",
            "by": "doi",
            "resource": ""
        }
    }
}

# Second pattern: refs_agg
refs_agg = ap_schema.get_field("refs_agg")
refs_agg.title = "Aggregated references"
refs_agg.description = "Sum of all incoming references for this DOI"
refs_agg.type = "integer"
refs_agg.missing_values = ["", "n/a", "NaN"]
refs_agg.mic = {
    "type": "pattern",
    "prov": {
        "operation": {
            "type": "aggregation",
            "by": "doi",
            "input": ""
        }
    }
}

# Add third pattern which is an average

SyntaxError: invalid syntax (<ipython-input-7-41b5306200e4>, line 16)

In [ ]:
# Resource specs
resource_desc = {
    "profile": "pattern",
    "name": "Article-level metrics",
    "data": data_dir / "article_patterns.csv",
    "schema": ap_schema
}

## References

Nakassis, C. V. (2013). Citation and Citationality. Signs and Society, 1(1), 51–77. https://doi.org/10.1086/670165